In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

ROLL_NUM = 102316108

In [31]:
df=pd.read_csv('data.csv', encoding='latin1')

X=df['no2'].dropna().values
X[:5]

C:\Users\nj323\AppData\Local\Temp\ipykernel_29776\1640709863.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data.csv', encoding='latin1')


array([17.4,  7. , 28.5, 14.7,  7.5])

In [32]:
X.dtype

dtype('float64')

In [33]:
X.astype(np.float32)

array([17.4,  7. , 28.5, ..., 45. , 50. , 46. ], dtype=float32)

In [34]:
a_r = 0.5*(ROLL_NUM*7)
b_r = 0.3*((ROLL_NUM%5) + 1)

print(a_r)
print(b_r)

358106378.0
1.2


In [35]:
z = X + a_r*np.sin(b_r*X)
Z_tensor=torch.tensor(z).view(-1,1)

Z_tensor

tensor([[ 3.2094e+08],
        [ 3.0604e+08],
        [ 1.2532e+08],
        ...,
        [-2.0011e+08],
        [-1.0915e+08],
        [-3.4931e+08]], dtype=torch.float64)

In [36]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(1,16),
            nn.ReLU(),
            nn.Linear(16,16),
            nn.ReLU(),
            nn.Linear(16,1),
        )
    
    def forward(self,x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid(),
        )
    
    def forward(self,x):
        return self.net(x)

In [37]:
g=Generator()
d=Discriminator()
critera=nn.BCELoss()

g_optimizer=optim.Adam(g.parameters(),lr=0.001)
d_optimizer=optim.Adam(d.parameters(),lr=0.001)

epochs=50
batch_size=64

In [38]:
for epoch in range(epochs):
    idx=np.random.randint(0,len(Z_tensor),batch_size)
    real_samples=Z_tensor[idx]

    real_labels=torch.ones(batch_size,1)
    fake_labels=torch.zeros(batch_size,1)

    noise=torch.randn(batch_size,1)
    fake_samples=g(noise)

    d_real=d(real_samples)
    d_fake=d(fake_samples.detach())

    d_loss=critera(d_real,real_labels)+critera(d_fake,fake_labels)

    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    noise=torch.randn(batch_size,1)
    fake_samples=g(noise)
    d_fake=d(fake_samples)

    g_loss=critera(d_fake,real_labels)

    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    if epoch%500==0:
        print(f'epoch {epoch} and D loss: {d_loss.item():.2f} and G loss: {g_loss.item():.2f}')

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
noise=torch.randn(10000,1)
z_fake=g(noise).detach().numpy().flatten()

In [ ]:
plt.hist(z,bins=50, density=True, alpha=0.6, label="real z")
plt.hist(z_fake,bins=50, density=True, alpha=0.6, label="gan z")

plt.xlabel('z')
plt.xlabel('density')
plt.title('pdf approx using gan')
plt.legend()
plt.show()